In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from timeit import default_timer

import paddle
from paddle.io import DataLoader, TensorDataset

paddle.seed(0)
np.random.seed(0)
paddle.set_device('gpu')

# Real-time design optimization 

During optimization, we start from initial design parameters
$(L_p = 100, x2 = -40, x3 = -30, h = 25.0) \mu m$, and update them using the BFGS algorithm to minimize the objective function ⟨Xup⟩ post-processed from the bacteria population predicted by Geo-FNO. When the optimization gets trapped in a local minimizer, the optimization restarts from an initial condition obtained by perturbing the recordedglobal minimizer with a random Gaussian noise sampled from $N(0, I)$. The proposed randomized BFGS algorithm guarantees the recorded-global minimizer monotonically decreases. 

In [ ]:
################################################################
# inverse optimization for 1d
################################################################
from catheter import *
model = FNO1d(64, 64, 100, input_channel=2, output_np=2001)
model.set_state_dict(paddle.load("/home/lunhao/Replication/AiScience/geofno-paddle/model/catheter_plain_length_model_1d1000.pdparams"))

epochs = 100

# constraints   
#               60 < L_p < 250
#               x1 = -0.5L_p 
#               
#               15 < x3 - x1 < L_p/2
#             -L_p < x2      < 0
#               20 < h       < 30
# def transfer(theta):
def transfer(theta):
    
    L_p = 60 + (250 - 60)/(1 + paddle.exp(theta[0]))
    x1 = -0.5*L_p
    x3 = x1  + 15 + (L_p/2 - 15)/(1 + paddle.exp(theta[2]))
    x2 = -L_p  + L_p/(1 + paddle.exp(theta[1]))
    h = 20   + (10)/(1 + paddle.exp(theta[3]))
    return L_p, x1, x2, x3, h


def inv_transfer(L_p, x2, x3, h):
    x1 = -0.5*L_p
    theta = np.zeros(4)
    theta[0] = np.log( (250 - 60)/(L_p - 60) - 1 )
    theta[1] = np.log( L_p/(x2 + L_p) - 1 )
    theta[2] = np.log( (L_p/2 - 15)/(x3 - x1  - 15) - 1 )
    theta[3] = np.log( 10/(h - 20 ) - 1 )
    return theta

L_p, x2, x3, h = 100.0, -40.0, -30.0, 25.0
theta0 =  inv_transfer(L_p, x2, x3, h) 
print("initialize : ", theta0)


bacteria_pred = []
L_x = 500
N_s = 2001
xx_mask = (paddle.linspace(1.0, 0, N_s) * (-L_x))

max_iter = 100


loss_min = paddle.to_tensor(np.inf)

loss_all = []
theta_min = np.copy(theta0)
theta_min_perturbed = np.copy(theta_min)


def saveplot(theta, out, savefig_name):
    bacteria_pred.append(out)
    L_p, x1, x2, x3, h = transfer(theta)
    mesh = x.detach().cpu().numpy()
    
    current_loss = (-paddle.sum(paddle.matmul(out, xx_mask))* L_x/N_s).item()
    
    print("L_p, x1, x2, x3, h  = ", L_p.item(), x1.item(), x2.item(), x3.item(), h.item())
    plt.figure(figsize=(4,4), dpi=1200)
    plt.title("Real-time design optimization")
    plt.plot(mesh[0, :, 0], mesh[0, :, 1], color="C1", label="Channel geometry")
    plt.plot(mesh[0, :, 0], 100-mesh[0, :, 1], color="C1")
    
    plt.plot(xx_mask.detach().cpu().numpy(), out.detach().cpu().numpy().T, "--*", color="C2", fillstyle='none', markevery=len(xx_mask)//10, label="Predicted bacteria distribution")
    
    plt.legend(loc="upper left")
    plt.ylim([-50,500])
    plt.xlabel(r"x")
    
    plt.text(-300,300,"Loss = "+r"$%3.1f$" %current_loss,\
          bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1})
    
    
    plt.tight_layout()
    plt.savefig(savefig_name)
    plt.show()
    
plot_id = 0

# first_figure
theta = paddle.to_tensor(theta_min_perturbed.astype(np.float32), stop_gradient=False)
L_p, x1, x2, x3, h = transfer(theta)  
x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
out = paddle.exp(model(x).squeeze())
saveplot(theta, out, "movie/design_iter" + str(plot_id).zfill(4) + ".png")
plot_id += 1

for ep in range(epochs):
    theta = paddle.to_tensor(theta_min_perturbed.astype(np.float32) , stop_gradient=False)
    optimizer = paddle.optimizer.LBFGS([theta], max_iter=max_iter, learning_rate=1.0, line_search_fn="strong_wolfe")
    
    t1 = default_timer()
    def loss_closure():
        L_p, x1, x2, x3, h = transfer(theta)
        x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
        optimizer.zero_grad()
        out = paddle.exp(model(x).squeeze())
        loss = -paddle.sum(paddle.matmul(out, xx_mask))* L_x/N_s
        loss.backward()
        loss_all.append(loss.item())
        return loss
    
    
    optimizer.step(loss_closure)

    t2 = default_timer()
 
    if ep%1==0:
        
        L_p, x1, x2, x3, h = transfer(theta)
        x, XC, YC = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
        out = paddle.exp(model(x).squeeze())
        loss = -paddle.sum(paddle.matmul(out, xx_mask))* L_x/N_s
        print(ep, t2 - t1,  "loss = ", loss.item(), "loss_min = ", loss_min.item(),)
 
        if loss < loss_min:
            
            theta_min = theta.detach().numpy()
            loss_min = loss
            
            
            saveplot(theta, out, "movie/design_iter"+ str(plot_id).zfill(4)+".png")
            plot_id += 1
            
        theta_min_perturbed = theta_min + np.random.normal(loc=0.0, scale=1.0, size=len(theta_min))
        


In [ ]:
fig = plt.figure(figsize=(8,3))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
plt.plot(loss_all[0:2000], "-", markersize=1)
ax.set_yticks([2e5, 4e5, 6e5, 8e5, 1e6])
ax.set_xticks([0, 500, 1000, 1500, 2000])
ax.set_yticklabels([2, 4, 6, 8, 10])
ax.set_ylim([2e5, 12e5])
minind = np.array(loss_all).argmin()
plt.scatter(minind, loss_all[minind], color="red", s=2)
fig.savefig("optimization-loss.pdf")

In [ ]:
import numpy

# All data design results
PATH = "/groups/esm/dzhuang/Catheter/allparam/length/"
INPUT_X = PATH+"x_1d_structured_mesh.npy"
INPUT_Y = PATH+"y_1d_structured_mesh.npy"
INPUT_para = PATH+"data_info.npy"
OUTPUT = PATH+"density_1d_data.npy"
n_data = 3000
inputX_raw = np.load(INPUT_X)[:, 0:n_data]
inputY_raw = np.load(INPUT_Y)[:, 0:n_data]
inputPara_raw = np.load(INPUT_para)[:, 0:n_data]
output_raw = np.load(OUTPUT)[:, 0:n_data]


# nx ny
L_x , N_s = 500.0, 2001

################################################################
# load data and data normalization
################################################################
inputX = inputX_raw[:, 0::3]
inputY = inputY_raw[:, 0::3]
inputPara = inputPara_raw[:, 0::3]
output = (output_raw[:, 0::3] + output_raw[:, 1::3] + output_raw[:, 2::3])/ 3.0


n_data = inputX.shape[1]
all_loss = np.zeros(n_data)


for i in range(n_data):
    
    sample, uf, L_p, x1, x2, x3, h = inputPara[:, i]
    xx_mask = numpy.linspace(1.0, 0, N_s) * (-L_x)
    all_loss[i] = -np.dot(output[:, i], xx_mask) * L_x/N_s

    

loss_min_ind = np.argmin(all_loss)
loss_min = all_loss[loss_min_ind]
sample, uf, L_p, x1, x2, x3, h = inputPara[:, loss_min_ind]

model = torch.load("catheter_plain_length_model_1d1000", map_location=device)
L_p, x2, x3, h =  torch.tensor(L_p, dtype=torch.float), torch.tensor(x2, dtype=torch.float), torch.tensor(x3, dtype=torch.float), torch.tensor(h, dtype=torch.float)

X_Y, X, Y = catheter_mesh_1d_total_length(L_x, L_p, x2, x3, h, N_s)
out = torch.exp(model(X_Y).squeeze()).detach().cpu().numpy()


print(inputPara[:, loss_min_ind])
print("Sample id = ", sample)
print("Minimum loss = ", loss_min, "L_p, x2, x3, h = ", L_p, x2, x3, h)

xx_mask = numpy.linspace(1.0, 0, N_s) * (-L_x)
print("Predicted loss: ", -np.dot(out, xx_mask) * L_x/N_s)

density = output[:, loss_min_ind]
mesh_X = np.flip(inputX[:, loss_min_ind])
mesh_Y = np.flip(inputY[:, loss_min_ind])

plt.figure()
plt.plot(mesh_X, mesh_Y, color="C1", label="Channel geometry")
plt.plot(mesh_X, 100-mesh_Y, color="C1")
plt.plot(xx_mask, density, "--o", color="red", markevery=len(xx_mask)//10, fillstyle='none', label="Reference")     
plt.plot(xx_mask, out, "--*", color="C2", fillstyle='none', markevery=len(xx_mask)//10, label="Predicted bacteria distribution")
plt.legend()


In [ ]:
def numpy_catheter_mesh_1d_total_length(L_x, L_p, x1, x2, x3, h, N_s):
    # between [-L_p, 0]
    
    n_periods = np.floor(L_x / L_p)
    L_x_last_period = L_x - n_periods*L_p
    L_p_s = ((x1 + L_p) + (0 - x3) + np.sqrt((x2 - x1)**2 + h**2) + np.sqrt((x3 - x2)**2 + h**2))
    L_s = L_p_s*n_periods + numpy_Lx2length(L_x_last_period, L_p, x1, x2, x3, h)
    
    # from 0
    d_arr = np.linspace(0, L_s, N_s)
    period_arr = np.floor(d_arr / L_p_s)
    d_arr -= period_arr * L_p_s
    
    xx = np.zeros(N_s) 
    yy = np.zeros(N_s)
    for i in range(N_s):
        xx[i], yy[i] = numpy_d2xy(d_arr[i], L_p, x1, x2, x3, h)
        xx[i] -= period_arr[i]*L_p
    return xx, yy

In [ ]:
# Draw initial guess and final designs 

L_x, L_p, x2, x3, h = 500, 100.0, -40.0, -30.0, 25.0
x1 = -0.5*L_p
N_s = 2001
X0,Y0 = numpy_catheter_mesh_1d_total_length(L_x, L_p, x1, x2, x3, h, N_s)
plt.figure(figsize=(6, 2))
plt.plot(X0, 100-Y0, color="#464546", linewidth=4.0)
plt.plot(X0, Y0, color="#464546", linewidth=4.0)
plt.axis('off'),
plt.xlim([-300, 0])
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.savefig("initial_guess.pdf")

L_p, x1, x2, x3, h = 62.2564697265625,  -31.12823486328125, -11.565677642822266, -15.864848136901855, 29.99827766418457
x1 = -0.5*L_p
N_s = 2001
X0,Y0 = numpy_catheter_mesh_1d_total_length(L_x, L_p, x1, x2, x3, h, N_s)
plt.figure(figsize=(6, 2))
plt.plot(X0, 100-Y0, color="#D87CA9", linewidth=4.0)
plt.plot(X0, Y0, color="#D87CA9", linewidth=4.0)
plt.axis('off'),
plt.xlim([-300, 0])
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.savefig("final_guess.pdf")





In [ ]:
import glob
# Design validation

train_data_folder = "optimal_design_data"
t = 9


N_s, L_x = 2001, 500
bw_method = 1e-1
xx = np.linspace(-L_x, 0.0, N_s)
xx_mask = (torch.linspace(1.0, 0, N_s) * (-L_x)).to(device)

samples = [0]
x_mesh, y_mesh = np.zeros((len(samples), N_s)), np.zeros((len(samples), N_s))
density_1d_data_all = np.zeros((len(samples), N_s, 3))
density_1d_data = np.zeros((len(samples), N_s))
L_p, x1, x2, x3, h = 62.2564697265625,  -31.12823486328125, -11.565677642822266, -15.864848136901855, 29.99827766418457
# L_p, x1, x2, x3, h = 62.32211685180664, -31.16105842590332, -11.533348083496094, -16.148033142089844, 29.966632843017578

for i in range(len(samples)):
    
    x_mesh[i,:], y_mesh[i,:] = numpy_catheter_mesh_1d_total_length(L_x, L_p, x1, x2, x3, h, N_s)
    
    file_names = glob.glob(train_data_folder+"/tri*")
    for file_name in file_names:
        
        print(file_name)
        uf  = np.float64(  file_name[file_name.find("uf") + len("uf"):  file_name.find("alpha")]  )
        # sample is from 1 - 1000
        j = 0
        if uf < (5.0 + 10.0)/2:
            j = 0
        elif uf < (10. + 15.0)/2:
            j = 1
        elif uf < (15. + 20.0)/2:
            j = 2
        else:
            print("error! uf = ", uf)
            
        
        # preprocee density
        hf = h5py.File(file_name, "r")
        x_b = hf["config"][str(t+1)]["x"][:]
        y_b = hf["config"][str(t+1)]["y"][:]

        if(min(x_b) < -L_x):
            print("warning: bacteria out of the domain. file_name = ", file_name, " loc = ", min(x_b), " end point = ", -L_x)

        

        bacteria_1d_data = x_b[np.logical_and(x_b <= 0 , x_b >= -L_x)]
        n_particle = len(bacteria_1d_data)
        kernel = stats.gaussian_kde(bacteria_1d_data, bw_method = bw_method)
        density_1d_data_all[i, :, j] = kernel(xx)*n_particle
    
    density_1d_data[i, :] = np.mean(density_1d_data_all[i, :, :], axis=1)   
    
    
    
    ref_loss = -np.sum(np.matmul(density_1d_data[i, :], xx))* L_x/N_s
    x, XC, YC = catheter_mesh_1d_total_length(L_x, torch.tensor(L_p), torch.tensor(x2), torch.tensor(x3), h, N_s)
    out = torch.exp(model(x).squeeze())
    fno_loss = -torch.sum(torch.matmul(out, xx_mask))* L_x/N_s
    print("fno_loss = ", fno_loss.item(), "ref_loss = ", ref_loss)
       
    
    
    
    xx_mask = np.linspace(1.0, 0, N_s) * (-L_x)
    #######################################################################
    plt.figure(figsize=(4.5, 3))
    plt.plot(xx_mask, bacteria_pred[0].detach().cpu().numpy(), color="#464546", linewidth=2.0, label="initial design")
    plt.semilogy(xx_mask, bacteria_pred[-1].detach().cpu().numpy(), color="#D87CA9", linewidth=2.0, label="optimal design")
    plt.semilogy(xx_mask, density_1d_data[i, :], "--",  color="green", linewidth=2.0, label="reference")
    plt.xlim([-300, 0])
    plt.ylim([0.2, 1e3])
    
    plt.tick_params(labelsize=0) 
#     plt.legend()
#     plt.title("Bacteria Distribution")
#     plt.xlabel(r"x")
    plt.savefig("bacteria_population_semilogy.pdf")


